In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer

from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn import metrics
from sklearn.metrics import RocCurveDisplay

In [2]:
base = pd.read_csv("basePreProcessedAllAbFinal.csv")

y_data = base["status"]
x_data = base.drop(["status"], axis=1)

In [3]:
x_data.info()
y_data.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3589 entries, 0 to 3588
Columns: 170 entries, Molecular Weight to UFF Energy
dtypes: float64(82), int64(88)
memory usage: 4.7 MB


status
1.0    1914
0.0    1675
Name: count, dtype: int64

In [4]:
def accuracy_per_class(y_test, y_pred):
    """
    Calculate accuracy per class.

    Parameters
    ----------
    y_test : numpy.ndarray
        True labels.
    y_pred : numpy.ndarray
        Predicted labels.

    Returns
    -------
    acc_pos: float
        Accuracy for the positive class.
    acc_neg: float
        Accuracy for the negative class.
    acc: float
        Average accuracy.

    """
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

    #Fix - Problema de divisão por 0 em thresholds que colocam todas as instâncias como positivas ou negativas tendo TN e FN como 0 ou TP e FP como 0
    if (tp + fp) == 0:
        acc_pos = 0
    else:
        acc_pos = tp / (tp + fp)

    if (tn + fn) == 0:
        acc_neg = 0
    else:
        acc_neg = tn / (tn + fn)
    #Fix

    acc = (tp + tn) / (tp + tn + fp + fn)
    return acc_pos, acc_neg, acc

def auc_eval(y_test, y_pred, positive = 1):
    """
    Calculate AUC per class.

    Parameters
    ----------
    y_test : numpy.ndarray
        True labels.
    y_pred : numpy.ndarray
        Predicted labels.
    positive : int
        Index of positive class.

    Returns
    -------
    auc : float
        Area under the ROC curve.

    """
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=positive)
    return metrics.auc(fpr, tpr)

def roc_calc_viz_pred(y_true, y_pred):
    viz = RocCurveDisplay.from_predictions(
                            y_true,
                            y_pred
                        )

    return viz.fpr, viz.tpr, viz.roc_auc

def positive_negative_rate(y_test, y_pred):
    
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    specificity = tn / (tn+fp)
    sensibivity = tp / (tp+fn)
    accuracy = (tp + tn)/(tp + tn + fp + fn)
    return round(sensibivity, 5), round(specificity, 5), round(accuracy,5) 

In [11]:
random_state = 1
folds = 10
rangeThreshold = [0.1, 0.81, 0.1]
cvResults = []
countFold = 1

thresholdList = []
for thresh in np.arange(start=rangeThreshold[0], stop=rangeThreshold[1], step=rangeThreshold[2]):
    thresholdList.append(thresh)


scaler = StandardScaler()
#imputer = KNNImputer(n_neighbors=5)
imputer = SimpleImputer(strategy='mean')

#classifier = KNeighborsClassifier()
#classifier = KNeighborsClassifier(weights='distance')
#classifier = SVC(random_state=random_state, probability=True)
#classifier = LogisticRegression(random_state=random_state)
classifier = DecisionTreeClassifier(random_state=random_state)
#classifier = RandomForestClassifier(random_state=random_state, criterion='gini', max_features=13, n_estimators=1275, max_depth=32)
#classifier = XGBClassifier(random_state=random_state)

kf = KFold(n_splits=folds, shuffle=True, random_state=random_state) 

for train_index, test_index in kf.split(base): #10-fold cross-validation (90-10??? como garantir?)

    print('-' * 30)
    print(f"Fold: {countFold}")
    countFold += 1

    x_train, x_test = x_data.iloc[train_index], x_data.iloc[test_index]
    y_train, y_test = y_data.iloc[train_index], y_data.iloc[test_index]

    #Mean Imputation
    #x_train = imputer.fit_transform(x_train) #Uncomment this for pipeline 2
    #x_test = imputer.transform(x_test) #Uncomment this for pipeline 2

    # Standardize the sets using the same scaler
    #x_train = scaler.fit_transform(x_train)
    #x_test = scaler.transform(x_test)

    # Apply PCA to training data to retain 95% variance
    #pca = PCA(n_components=0.95) #Comment this for pipeline 1_semPCA
    #x_train = pca.fit_transform(x_train) #Comment this for pipeline 1_semPCA

    # Apply the same PCA transformation to the test set
    #x_test = pca.transform(x_test) #Comment this for pipeline 1_semPCA

    # SMOTE augmentation on the PCA-reduced training set
    #smote = SMOTE(k_neighbors=3, random_state=random_state)
    #x_train, y_train = smote.fit_resample(x_train, y_train)

    #x_train = pd.DataFrame(x_train)
    #x_test = pd.DataFrame(x_test)

    results = {'model_name': [], 'F1':[], 'F1-Macro':[], 'ROC':[], 'acc-class-1':[], 'acc-class-2':[], 'ACC':[], 'ACC-Balanced':[], 'TPR': [], 'FPR':[], 'AUC': [], 'THRE': [], 'SEN': [], 'SPE': [], 'MCC': []}

    classifier.fit(x_train, y_train)
    y_pred_proba = classifier.predict_proba(x_test)[:, 1]

    for thresh in np.arange(start=rangeThreshold[0], stop=rangeThreshold[1], step=rangeThreshold[2]):
        print(f'Threshold with {thresh}')
        
        y_pred = (y_pred_proba >= thresh).astype(int)
                
        acc_pos, acc_neg, acc = accuracy_per_class(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average='binary')
        f1_macro = f1_score(y_test, y_pred, average='macro') #Inclusão do F1 macro
        auc = auc_eval(y_test, y_pred)
        acc_class = [acc_pos, acc_neg]
        viz_fpr = 0
        viz_tpr = 0
        viz_auc = 0 # No pipeline original: viz_fpr, viz_tpr, viz_auc = roc_calc_viz_pred(y_test, y_pred_proba) -> Removido pois não são métricas utilizadas e por estarem imprimindo os gráficos
        sen,spe,_ = positive_negative_rate(y_test,y_pred)
        mcc = matthews_corrcoef(y_test, y_pred) #Inclusão do MCC
        balanced_acc = balanced_accuracy_score(y_test, y_pred) #Inclusão do balanced_accuracy

        results['model_name'].append('Decision Tree')
        results['acc-class-1'].append(acc_pos)
        results['acc-class-2'].append(acc_neg)
        results['ACC'].append(acc)
        results['ACC-Balanced'].append(balanced_acc)
        results['F1'].append(f1)
        results['F1-Macro'].append(f1_macro)
        results['ROC'].append(auc)
        results['FPR'].append(viz_fpr)
        results['TPR'].append(viz_tpr)
        results['AUC'].append(viz_auc)
        results['THRE'].append(thresh)
        results['SPE'].append(spe)
        results['SEN'].append(sen)
        results['MCC'].append(mcc)

    cvResults.append(results) #Cada lista terá multiplos valores pois executa sobre varios thresholds testados (teria um valor só se fosse um único threshold). Assim, cvResults é um vetor de dicts

cvFinalMetrics = {"ACC-Balanced":[], "ACC":[], 'F1':[], 'F1-Macro':[], 'ROC':[], 'MCC': [], 'acc-class-1':[], 'acc-class-2':[], 'SEN': [], 'SPE': []} #Removido 'model_name', 'THRE', 'TPR', 'FPR' e 'AUC'. 'TPR', 'FPR' e 'AUC' foram removidos pois são vetores com mais de 500 valores, correspondendo a pontos da curva ROC discretizada, permite compor visualmente a curva ROC. 'THRE' foi removido por ser o Threshold. Foi reordenado para facilitar a adição na planilia
    
#Calculate average for each metric and each threshold
for metric in cvFinalMetrics:
    temp_metric = []

    for i in range(folds): #Pega o vetor de valores por fold da métrica em questão (são vetores pois temos um valor por threshold) 
        temp_metric.append(cvResults[i][metric])
    
    for i in range(len(thresholdList)): #Calcula a média por threshold
        aux = []
        
        for i2 in range(folds):
            aux.append(temp_metric[i2][i])
        
        cvFinalMetrics[metric].append(np.average(aux))

#Print results
for metric in cvFinalMetrics:
    print(f"--{metric}")
    for i,thres in enumerate(thresholdList):
        print(f"-Threshold {thres}: {cvFinalMetrics[metric][i]}")
    
results_df = {"Threshold": thresholdList}
results_df.update(cvFinalMetrics)
results_df = pd.DataFrame(results_df)
print('-' * 15 + "FDA" + '-' * 15)
print(results_df)
results_df.to_csv(f"FDA_BaseModels_DT_results.csv", index=False)

------------------------------
Fold: 1
Threshold with 0.1
Threshold with 0.2
Threshold with 0.30000000000000004
Threshold with 0.4
Threshold with 0.5
Threshold with 0.6
Threshold with 0.7000000000000001
Threshold with 0.8
------------------------------
Fold: 2
Threshold with 0.1
Threshold with 0.2
Threshold with 0.30000000000000004
Threshold with 0.4
Threshold with 0.5
Threshold with 0.6
Threshold with 0.7000000000000001
Threshold with 0.8
------------------------------
Fold: 3
Threshold with 0.1
Threshold with 0.2
Threshold with 0.30000000000000004
Threshold with 0.4
Threshold with 0.5
Threshold with 0.6
Threshold with 0.7000000000000001
Threshold with 0.8
------------------------------
Fold: 4
Threshold with 0.1
Threshold with 0.2
Threshold with 0.30000000000000004
Threshold with 0.4
Threshold with 0.5
Threshold with 0.6
Threshold with 0.7000000000000001
Threshold with 0.8
------------------------------
Fold: 5
Threshold with 0.1
Threshold with 0.2
Threshold with 0.30000000000000004
